In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn.functional as F
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
train_data = datasets.CIFAR10(
  root="datasets",
  train=True,
  download=True,
  transform=transform,
  target_transform=None
)
test_data = datasets.CIFAR10(
  root="datasets",
  train=False,
  download=True,
  transform=transform,
  target_transform=None
)

100%|██████████| 170498071/170498071 [00:03<00:00, 43994575.69it/s]


Extracting datasets/cifar-10-python.tar.gz to datasets
Files already downloaded and verified


In [4]:
batch_size = 4
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
def acc(y_pred, y_true):
  if len(y_pred) != len(y_true):
    print("Error: y_pred and y_true have different lengths. Aborting")
  return torch.eq(y_pred, y_true).sum().item() / len(y_pred)

In [14]:
class AnotherLeNet5LikeModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_block = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
      nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
      nn.ReLU(),
      nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
    )

    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(16*5*5, 120),
      nn.ReLU(),
      nn.Linear(120, 84),
      nn.ReLU(),
      nn.Linear(84, 10),
    )


  def forward(self, x):
    return self.classifier(
      self.conv_block(x)
    )

model = AnotherLeNet5LikeModelV2().to(device)
print(model)
print(next(model.parameters()).device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

AnotherLeNet5LikeModelV2(
  (conv_block): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): ReLU()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): ReLU()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)
cuda:0


In [15]:
model.train()
epochs = 6
for epoch in tqdm(range(epochs)):
  print(f"epoch {epoch} ---")
  epoch_loss = 0

  for i, (image, label) in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):

    logits = model(image.to(device))

    loss = loss_fn(logits, label.to(device))

    epoch_loss += loss
    if i % 1500 == 0:
      print(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("---")
  print("epoch loss: ", epoch_loss.item() / len(train_dataloader))


model.eval();

  0%|          | 0/6 [00:00<?, ?it/s]

epoch 0 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

2.2860751152038574
1.883562445640564
2.3318231105804443
1.9859623908996582
2.2835466861724854
1.446353554725647
3.0691258907318115
1.9186092615127563
2.1625845432281494
---
epoch loss:  1.7760634375
epoch 1 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

...

0.9759892225265503
1.5562264919281006
2.340175151824951
1.0449726581573486
1.5863747596740723
1.1206963062286377
0.564643144607544
1.0369162559509277
0.7589882612228394
---
epoch loss:  1.196031640625
epoch 4 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

0.7266194820404053
0.9535952210426331
1.1439437866210938
2.3038573265075684
1.543155312538147
0.6341149806976318
0.9775421619415283
0.13701534271240234
0.7519926428794861
---
epoch loss:  1.11589734375
epoch 5 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

0.84233158826828
0.3698819577693939
0.7155264616012573
0.19744420051574707
1.0341289043426514
0.6292532682418823
1.1761013269424438
1.3730911016464233
0.9148853421211243
---
epoch loss:  1.0531028125


In [16]:
with torch.inference_mode():
  avg_acc = 0
  avg_loss = 0
  for i, (image, label) in enumerate(tqdm(test_dataloader)):
    label = label.to(device)

    logits = model(image.to(device))
    loss = loss_fn(logits, label)
    accuracy = acc(torch.argmax(logits, dim=1), label)

    avg_loss += loss
    avg_acc += accuracy
    if i % 300 == 0:
      print(f"{i:5} | {accuracy*100:7.2f}% | {loss.item():.5f}")


  print(avg_acc / len(test_dataloader)*100, avg_loss.item() / len(test_dataloader))

  0%|          | 0/2500 [00:00<?, ?it/s]

    0 |  100.00% | 0.52918
  300 |   75.00% | 0.41375
  600 |   25.00% | 1.37279
  900 |   25.00% | 1.64587
 1200 |   50.00% | 1.09688
 1500 |  100.00% | 0.64886
 1800 |   25.00% | 2.38781
 2100 |   50.00% | 1.06193
 2400 |   50.00% | 1.50814
58.81999999999999 1.1766951171875


Old model with smaller batchsize

In [18]:
class NewConvModelV1(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden_units = 10
    self.classes = 10

    self.conv_block = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=self.hidden_units, kernel_size=(3,3), stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(self.hidden_units, self.hidden_units, (3, 3), stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2)),
    )


    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(2560, self.classes)
    )


  def forward(self, x):
    return self.classifier(
        self.conv_block(x)
    )


model = NewConvModelV1().to(device)
print(model)
print(next(model.parameters()).device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

NewConvModelV1(
  (conv_block): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=10, bias=True)
  )
)
cuda:0


In [19]:
model.train()
epochs = 4
for epoch in tqdm(range(epochs)):
  print(f"epoch {epoch} ---")
  epoch_loss = 0

  for i, (image, label) in enumerate(tqdm(train_dataloader, total=len(train_dataloader))):

    logits = model(image.to(device))

    loss = loss_fn(logits, label.to(device))

    epoch_loss += loss
    if i % 1500 == 0:
      print(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("---")
  print("epoch loss: ", epoch_loss.item() / len(train_dataloader))


model.eval();

  0%|          | 0/4 [00:00<?, ?it/s]

epoch 0 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

2.2729332447052
2.1018009185791016
1.543776273727417
1.2271828651428223
1.9305293560028076
1.8017122745513916
1.099902868270874
1.1661165952682495
1.876471996307373
---
epoch loss:  1.47245359375
epoch 1 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

1.3277983665466309
0.9171934723854065
1.281090259552002
0.6019275784492493
1.8341144323349
1.1669981479644775
0.5314083099365234
1.5367307662963867
0.2897513806819916
---
epoch loss:  1.164630859375
epoch 2 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

0.8448705673217773
1.298084020614624
0.31321069598197937
1.065359115600586
1.2213281393051147
0.38325467705726624
0.8318032026290894
2.4907076358795166
1.797843098640442
---
epoch loss:  1.072953984375
epoch 3 ---


  0%|          | 0/12500 [00:00<?, ?it/s]

0.6181460618972778
0.9791085720062256
1.4356173276901245
1.0780532360076904
2.558105230331421
1.3753085136413574
0.9238529801368713
3.138360023498535
0.3243173658847809
---
epoch loss:  1.01704984375


In [20]:
with torch.inference_mode():
  avg_acc = 0
  avg_loss = 0
  for i, (image, label) in enumerate(tqdm(test_dataloader)):
    label = label.to(device)

    logits = model(image.to(device))
    loss = loss_fn(logits, label)
    accuracy = acc(torch.argmax(logits, dim=1), label)

    avg_loss += loss
    avg_acc += accuracy
    if i % 300 == 0:
      print(f"{i:5} | {accuracy*100:7.2f}% | {loss.item():.5f}")


  print(avg_acc / len(test_dataloader)*100, avg_loss.item() / len(test_dataloader))

  0%|          | 0/2500 [00:00<?, ?it/s]

    0 |   75.00% | 0.53834
  300 |   50.00% | 1.71380
  600 |   50.00% | 1.05719
  900 |    0.00% | 2.12770
 1200 |   75.00% | 0.77822
 1500 |   75.00% | 0.45799
 1800 |   25.00% | 1.89498
 2100 |   25.00% | 1.48455
 2400 |   25.00% | 3.19205
60.129999999999995 1.15742509765625


In [21]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.to(device))
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels.to(device), predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 53.3 %
Accuracy for class: car   is 65.8 %
Accuracy for class: bird  is 48.1 %
Accuracy for class: cat   is 51.2 %
Accuracy for class: deer  is 44.3 %
Accuracy for class: dog   is 57.9 %
Accuracy for class: frog  is 54.1 %
Accuracy for class: horse is 67.9 %
Accuracy for class: ship  is 80.6 %
Accuracy for class: truck is 78.1 %
